## Import of the raw data and preparation to further analyses
This notebook imports the original raw data, creates the unified dataframes and saves it in serialized files.

In [1]:
import sys
print("Python version: {}".format(sys.version))

Python version: 3.11.5 | packaged by Anaconda, Inc. | (main, Sep 11 2023, 13:26:23) [MSC v.1916 64 bit (AMD64)]


In [2]:
import pandas as pd
import datetime

path = "../data/" 
output_path = path+"prepared/"

In [3]:
def reformat_time(year,day):
    "changes day of the year and yer data to date"
    return datetime.datetime.strptime(str(year)+" "+str(day), '%Y %j').strftime('%Y.%m.%d')

reformat_time(2001,145) # print to check

'2001.05.25'

### Importing modelling errors
File `error-stat-10-years.xls` contains the predictions from the 2D shallow water equation (SWE-2D) and Baltic Sea observations.
Data is present in multiple sheets named with the year of observations/predictions.

In [4]:
# creating a list of sheet names
sheets = ["20"+ ("0"+str(i) if i<10 else str(i)) for i in range(8,18)]
print(sheets)

['2008', '2009', '2010', '2011', '2012', '2013', '2014', '2015', '2016', '2017']


In [5]:
tmp = []
for sheet in sheets:
    df_error = pd.read_excel(path + "error-stat-10-years.xls",sheet_name=sheet,usecols=[0,1,2])
    df_error["year"] = int(sheet)
    tmp.append(df_error)
    
# concatenation of the data frames
df_error = pd.concat(tmp)

cnames = list(df_error.columns)
cnames[0] = "t [day]"
cnames[1] = "predictions"
cnames[2] = "observations"
df_error.columns = cnames

df_error["t [day]"] += 1
df_error.index = pd.Index(range(0,len(df_error)))
df_error["date"] = pd.to_datetime(list(map(lambda x: reformat_time(x[0],x[1]),zip(df_error["year"],df_error["t [day]"]))))
df_error["error"] = df_error["observations"] - df_error["predictions"]

df_error = df_error[["date", "observations", "predictions"]] 
df_error.head(3)

,date,observations,predictions
0,2008-01-01,0.246,0.25
1,2008-01-02,0.223,0.18
2,2008-01-03,0.056,0.00


Checking if there are any missing or improper values.

In [6]:
df_error.isna().sum()

date            0
observations    0
predictions     0
dtype: int64

Exporting to serialized format and testing it.

In [7]:
df_error.to_pickle(output_path+"df_error.pkl")

In [8]:
df_error_test = pd.read_pickle(output_path+"df_error.pkl")
df_error_test.head(3)

,date,observations,predictions
0,2008-01-01,0.246,0.25
1,2008-01-02,0.223,0.18
2,2008-01-03,0.056,0.00


### Importing water stage data

#### Hel station

In [9]:
df_hel = pd.read_csv(path + "Hel_2008_2017.txt",sep="\s+",parse_dates=[['date', 'time']])
df_hel.head(3)

,date_time,stage
0,2008-01-01 00:10:00,527
1,2008-01-01 00:30:00,527
2,2008-01-01 03:20:00,526


In [10]:
df_hel.to_pickle(output_path+"df_hel.pkl")

In [11]:
df_hel_test = pd.read_pickle(output_path+"df_hel.pkl")
df_hel_test.head(3)

,date_time,stage
0,2008-01-01 00:10:00,527
1,2008-01-01 00:30:00,527
2,2008-01-01 03:20:00,526


#### Tolkmicko station

In [12]:
df_tol = pd.read_csv(path + "Tol_2008_2017.txt",sep="\s+",parse_dates=[['date', 'time']])
df_tol.head()

,date_time,stage
0,2008-01-01 01:40:00,534
1,2008-01-01 05:50:00,533
2,2008-01-01 07:20:00,532
3,2008-01-01 09:00:00,532
4,2008-01-01 10:30:00,532


In [13]:
df_tol.to_pickle(output_path+"df_tol.pkl")

In [14]:
df_tol_test = pd.read_pickle(output_path+"df_tol.pkl")
df_tol_test.head(3)

,date_time,stage
0,2008-01-01 01:40:00,534
1,2008-01-01 05:50:00,533
2,2008-01-01 07:20:00,532


### Wind and water stage data
Wind speed and direction is given in `wyn-YYYY.xlsx` files in `Wiatr` sheets, where `YYYY` denotes year in four digit format.
Daily water stage data at Tolkmicko and Hel are present in the same files, but in worksheets `Stany`.

In [15]:
# creating a list of file names
years = ["20"+ ("0"+str(i) if i<10 else str(i)) for i in range(8,18)]
#print(years)

fn = "wyn-"  # filename core

#### Wind data

In [16]:
tmp = []
for yr in years:
    df_wind = pd.read_excel(path + fn + str(yr) + ".xlsx",sheet_name="Wiatr")
    tmp.append(df_wind)
    
# concatenation of the data frames
df_wind = pd.concat(tmp)

df_wind["date_time"] = df_wind["day"] + pd.to_timedelta(list(map(lambda h:'0 days {}:00:00'.format(h),df_wind["hour"])))

df_wind = df_wind[["date_time","AL [st]", "W [m/s]"]]

df_wind.head()

,date_time,AL [st],W [m/s]
0,2008-01-01 00:00:00,151.83,2.50
1,2008-01-01 01:00:00,151.83,2.50
2,2008-01-01 02:00:00,155.17,2.78
3,2008-01-01 03:00:00,161.50,2.45
4,2008-01-01 04:00:00,142.00,2.75


In [17]:
df_wind.to_pickle(output_path+"df_wind.pkl")

In [18]:
df_wind_test = pd.read_pickle(output_path+"df_wind.pkl")
df_wind_test.head(3)

,date_time,AL [st],W [m/s]
0,2008-01-01 00:00:00,151.83,2.50
1,2008-01-01 01:00:00,151.83,2.50
2,2008-01-01 02:00:00,155.17,2.78


#### Water stage data

In [19]:
tmp = []
for yr in years:
    df_ws = pd.read_excel(path + fn + str(yr) + ".xlsx",sheet_name="Stany",usecols=[0,1,2,5,6,9,10])
    tmp.append(df_ws)
    
# concatenation of the data frames
df_water_stage = pd.concat(tmp)

df_water_stage.head()

,data,Hel (A),Tolkmicko (C),A [m npm],C [m npm],A-num,C-num
0,2008-01-01,524.8,532.6,0.248,0.246,0.25,0.25
1,2008-01-02,515.8,530.3,0.158,0.223,0.16,0.18
2,2008-01-03,495.1,513.6,-0.049,0.056,-0.05,0.00
3,2008-01-04,482.8,492.0,-0.172,-0.160,-0.17,-0.16
4,2008-01-05,470.1,485.3,-0.299,-0.227,-0.30,-0.33


In [20]:
df_water_stage.to_pickle(output_path+"df_water_stage.pkl")
df_water_stage_test = pd.read_pickle(output_path+"df_water_stage.pkl")
df_water_stage_test.head()

,data,Hel (A),Tolkmicko (C),A [m npm],C [m npm],A-num,C-num
0,2008-01-01,524.8,532.6,0.248,0.246,0.25,0.25
1,2008-01-02,515.8,530.3,0.158,0.223,0.16,0.18
2,2008-01-03,495.1,513.6,-0.049,0.056,-0.05,0.00
3,2008-01-04,482.8,492.0,-0.172,-0.160,-0.17,-0.16
4,2008-01-05,470.1,485.3,-0.299,-0.227,-0.30,-0.33


In [21]:
df_water_stage_test = pd.read_pickle(output_path+"df_water_stage.pkl")
df_water_stage_test.to_csv("water_stage.csv",sep=",",index=False)